# Optimisation via Simulation Tutorial

By Christine S.M Currie and Tom Monks

In [1]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [2]:
from ovs.toy_models import (custom_guass_bandit_problem,
                            BanditCasino, 
                            guassian_bandit_sequence)

## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [3]:
from ovs.indifference_zone import KN

In [4]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.05, 
        alpha=0.1, 
        n_0=5)

In [5]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[9]
allocations	[   8    9   10   11   12   13   39 1679  149 1680]
total reps	3610
means		[ 1.18770277  1.5962411   3.11797033  4.37426822  4.50532968  6.1088248
  6.96237079  7.98951005  8.94370307 10.01799484]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [6]:
from ovs.fixed_budget import OCBA

In [42]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=500, 
            delta=5, 
            n_0=5, 
            min=False)

call the `solve()` method to run the optimisation

In [43]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

best design:	9
allocations:	[  5   5   5  12   5   5   7  69 191 196]
total reps:	500
means:		[ -0.97  -1.55  -3.5   -3.98  -4.9   -6.2   -5.96  -8.02  -8.97 -10.  ]
vars:		[0.78 0.39 0.3  1.97 0.37 0.36 0.62 1.38 1.03 0.97]


# Evaluation of the methods

In [9]:
from ovs.evaluation import Experiment

In [10]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [11]:
results = exp.execute()

In [12]:
results.p_correct_selections

1.0

In [13]:
results.selections

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=int32)

In [14]:
results.correct_selections

10

In [15]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [16]:
results = exp.execute()

In [17]:
results.p_correct_selections

0.0

In [18]:
results.selections

array([99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 98, 99, 99, 99,
       99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99,
       99, 99, 99, 99, 99, 99, 98, 99, 99, 99, 99, 99, 99, 99, 99, 99],
      dtype=int32)

In [19]:
results.correct_selections

0

## Try different budgets

In [44]:
from ovs.evaluation import GridExperiment

In [48]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[1, 5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=1000)

In [49]:
results = exp.fit()

In [50]:
results

,n_designs,n_0,model,min,delta,budget,correct_selections,p_correct_selections
0,10,1,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,1,100,1000,1
1,10,5,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,1,100,1000,1
2,10,1,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,5,100,1000,1
3,10,5,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,5,100,1000,1
4,10,1,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,10,100,1000,1
5,10,5,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,10,100,1000,1
6,10,1,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,1,200,1000,1
7,10,5,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,1,200,1000,1
8,10,1,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,5,200,1000,1
9,10,5,<ovs.toy_models.BanditCasino object at 0x7fd19...,False,5,200,1000,1
